In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class MultiSkeleton2D3DAnimator:
    """
    Overlay multiple animated skeletons in ONE Plotly figure.
    Optionally show 2D keypoints on a side panel (animated, synced).

    Usage:
      anim = MultiSkeleton3DAnimator(fps=30, show_2d=True, y_axis_down=True)
      anim.add_sequence(X3d, edges=edges3d, color="blue", name="A", K2=K2d, edges2d=edges2d)
      anim.add_sequence(X3d_b, edges=edges3d, color="red",  name="B", K2=K2d_b)
      anim.fig.show()
    """

    def __init__(
        self,
        fps=30,
        marker_size=4,
        axis_pad_ratio=0.05,
        title="3D + 2D Multi-Skeleton Animation",
        show_2d=True,
        y_axis_down=True,   # True if 2D keypoints are in image coordinates (y down)
        two_d_aspect="equal"  # "equal" or "auto"
    ):
        self.fps = fps
        self.marker_size = marker_size
        self.axis_pad_ratio = axis_pad_ratio
        self.title = title
        self.show_2d = show_2d
        self.y_axis_down = y_axis_down
        self.two_d_aspect = two_d_aspect

        # Each seq dict can contain:
        # X (T,J,3), edges, color, name, marker_size, line_width,
        # K2 (T,J,2) optional, edges2d optional
        self.seqs = []

        # Build subplot layout: 3D left, 2D right
        if self.show_2d:
            self.fig = make_subplots(
                rows=1, cols=2,
                specs=[[{"type": "scene"}, {"type": "xy"}]],
                column_widths=[0.6, 0.4],
                horizontal_spacing=0.05,
                subplot_titles=("3D", "2D"),
            )
            base_layout = go.Layout(
                title=title,
                scene=dict(
                    aspectmode="cube",
                    xaxis=dict(autorange=False, title="X"),
                    yaxis=dict(autorange=False, title="Y"),
                    zaxis=dict(autorange=False, title="Z"),
                ),
                xaxis=dict(title="u"),
                yaxis=dict(title="v", autorange="reversed" if self.y_axis_down else True),
                uirevision="fixed",
                legend=dict(itemsizing="constant"),
            )
            self.fig.update_layout(base_layout)
        else:
            self.fig = go.Figure(
                layout=go.Layout(
                    title=title,
                    scene=dict(
                        aspectmode="cube",
                        xaxis=dict(autorange=False, title="X"),
                        yaxis=dict(autorange=False, title="Y"),
                        zaxis=dict(autorange=False, title="Z"),
                    ),
                    uirevision="fixed",
                    legend=dict(itemsizing="constant"),
                )
            )

        self._init_controls()

    def _init_controls(self):
        frame_duration_ms = int(1000 / max(self.fps, 1))
        self.fig.update_layout(
            updatemenus=[
                dict(
                    type="buttons",
                    showactive=False,
                    x=0.02, y=0.02,
                    xanchor="left", yanchor="bottom",
                    buttons=[
                        dict(
                            label="Play",
                            method="animate",
                            args=[
                                None,
                                dict(
                                    frame=dict(duration=frame_duration_ms, redraw=True),
                                    transition=dict(duration=0),
                                    fromcurrent=True,
                                ),
                            ],
                        ),
                        dict(
                            label="Pause",
                            method="animate",
                            args=[
                                [None],
                                dict(
                                    frame=dict(duration=0, redraw=False),
                                    transition=dict(duration=0),
                                    mode="immediate",
                                ),
                            ],
                        ),
                    ],
                )
            ],
            sliders=[],
        )

    @staticmethod
    def _bones_xyz(frame_xyz, edges):
        if not edges:
            return [], [], []
        xb, yb, zb = [], [], []
        for a, b in edges:
            xb += [frame_xyz[a, 0], frame_xyz[b, 0], None]
            yb += [frame_xyz[a, 1], frame_xyz[b, 1], None]
            zb += [frame_xyz[a, 2], frame_xyz[b, 2], None]
        return xb, yb, zb

    @staticmethod
    def _bones_uv(frame_uv, edges):
        if not edges:
            return [], []
        ub, vb = [], []
        for a, b in edges:
            ub += [frame_uv[a, 0], frame_uv[b, 0], None]
            vb += [frame_uv[a, 1], frame_uv[b, 1], None]
        return ub, vb

    def _recompute_scene_ranges(self):
        # fixed 3D ranges from ALL sequences combined
        if not self.seqs:
            return

        all_xyz = np.concatenate([s["X"].reshape(-1, 3) for s in self.seqs], axis=0)
        mins = np.nanmin(all_xyz, axis=0)
        maxs = np.nanmax(all_xyz, axis=0)
        center = (mins + maxs) / 2.0
        spans = (maxs - mins)
        max_span = float(np.nanmax(spans))

        pad = max_span * self.axis_pad_ratio
        half = max_span / 2.0 + pad

        xr = [center[0] - half, center[0] + half]
        yr = [center[1] - half, center[1] + half]
        zr = [center[2] - half, center[2] + half]

        self.fig.update_layout(
            scene=dict(
                xaxis=dict(range=xr, autorange=False),
                yaxis=dict(range=yr, autorange=False),
                zaxis=dict(range=zr, autorange=False),
                aspectmode="cube",
            )
        )

    def _recompute_2d_ranges(self):
        if not self.show_2d:
            return
        # fixed 2D ranges from ALL sequences that have K2
        all_2d = []
        for s in self.seqs:
            if s.get("K2") is not None:
                all_2d.append(s["K2"].reshape(-1, 2))
        if not all_2d:
            return

        all_uv = np.concatenate(all_2d, axis=0)
        mins = np.nanmin(all_uv, axis=0)
        maxs = np.nanmax(all_uv, axis=0)
        center = (mins + maxs) / 2.0
        spans = (maxs - mins)
        max_span = float(np.nanmax(spans))

        pad = max_span * self.axis_pad_ratio
        half = max_span / 2.0 + pad

        xr = [center[0] - half, center[0] + half]
        yr = [center[1] - half, center[1] + half]

        # For image coords you often want y reversed, but still a fixed numeric range.
        # Plotly's autorange="reversed" handles display direction.
        self.fig.update_xaxes(range=xr, autorange=False, row=1, col=2)
        # self.fig.update_yaxes(range=yr, autorange=False, row=1, col=2)
        self.fig.update_xaxes(range=xr, autorange=False, row=1, col=2)

        if self.y_axis_down:
            self.fig.update_yaxes(range=yr[::-1], autorange=False, row=1, col=2)
        else:
            self.fig.update_yaxes(range=yr, autorange=False, row=1, col=2)

        

        if self.two_d_aspect == "equal":
            # make 2D panel not stretched
            self.fig.update_yaxes(scaleanchor="x", scaleratio=1, row=1, col=2)

    def _rebuild_animation(self):
        """
        Rebuild frames so that every frame updates ALL traces.
        Per sequence trace layout:
          - 3D bones
          - 3D joints
          - 2D bones (optional if K2 provided)
          - 2D joints (optional if K2 provided)
        """
        if not self.seqs:
            return

        T_max = max(s["X"].shape[0] for s in self.seqs)

        frames = []
        for t in range(T_max):
            frame_data = []
            for s in self.seqs:
                X = s["X"]
                edges3d = s["edges"]
                K2 = s.get("K2", None)
                edges2d = s.get("edges2d", None) or edges3d  # default to same topology

                t_eff = min(t, X.shape[0] - 1)
                ft3 = X[t_eff]

                xb, yb, zb = self._bones_xyz(ft3, edges3d)

                # 3D traces
                frame_data.append(go.Scatter3d(x=xb, y=yb, z=zb))
                frame_data.append(go.Scatter3d(x=ft3[:, 0], y=ft3[:, 1], z=ft3[:, 2]))

                # 2D traces (if provided)
                if self.show_2d and (K2 is not None):
                    t_eff2 = min(t, K2.shape[0] - 1)
                    ft2 = K2[t_eff2]

                    ub, vb = self._bones_uv(ft2, edges2d)
                    frame_data.append(go.Scatter(x=ub, y=vb))
                    frame_data.append(go.Scatter(x=ft2[:, 0], y=ft2[:, 1]))

            frames.append(go.Frame(name=str(t), data=frame_data))

        self.fig.frames = tuple(frames)

        slider = dict(
            x=0.15, y=0.02,
            xanchor="left", yanchor="bottom",
            len=0.8,
            currentvalue=dict(prefix="t="),
            steps=[
                dict(
                    method="animate",
                    label=str(t),
                    args=[
                        [str(t)],
                        dict(
                            frame=dict(duration=0, redraw=True),
                            transition=dict(duration=0),
                            mode="immediate",
                        ),
                    ],
                )
                for t in range(T_max)
            ],
        )
        self.fig.update_layout(sliders=[slider])

    def add_sequence(
        self,
        X,
        edges=None,
        color="blue",
        name=None,
        line_width=4,
        marker_size=None,
        # NEW: 2D
        K2=None,             # (T,J,2) or None
        edges2d=None,
        color2d=None,        # optional different color for 2D
        line_width_2d=2,
        marker_size_2d=None,
    ):
        """
        Add another motion to overlay.
        X: (T,J,3)
        K2: optional (T,J,2) to show on the right panel
        """
        X = np.asarray(X)
        assert X.ndim == 3 and X.shape[2] == 3, "X must be shape (T, J, 3)"
        edges = edges or []

        if K2 is not None:
            K2 = np.asarray(K2)
            assert K2.ndim == 3 and K2.shape[2] == 2, "K2 must be shape (T, J, 2)"
            # allow different T, but J must match
            assert K2.shape[1] == X.shape[1], "K2 and X must have same number of joints (J)"

        marker_size = marker_size if marker_size is not None else self.marker_size
        marker_size_2d = marker_size_2d if marker_size_2d is not None else marker_size
        color2d = color2d if color2d is not None else color
        name = name or f"seq{len(self.seqs)}"

        self.seqs.append(
            dict(
                X=X, edges=edges, color=color, name=name,
                marker_size=marker_size, line_width=line_width,
                K2=K2, edges2d=(edges2d or edges),
                color2d=color2d, line_width_2d=line_width_2d, marker_size_2d=marker_size_2d
            )
        )

        # ---- Add initial traces (t=0) ----
        f0 = X[0]
        xb0, yb0, zb0 = self._bones_xyz(f0, edges)

        # 3D traces (left)
        if self.show_2d:
            self.fig.add_trace(
                go.Scatter3d(
                    x=xb0, y=yb0, z=zb0,
                    mode="lines",
                    line=dict(width=line_width, color=color),
                    name=f"{name}-bones",
                    showlegend=True,
                ),
                row=1, col=1
            )
            self.fig.add_trace(
                go.Scatter3d(
                    x=f0[:, 0], y=f0[:, 1], z=f0[:, 2],
                    mode="markers",
                    marker=dict(size=marker_size, color=color),
                    name=f"{name}-joints",
                    showlegend=True,
                ),
                row=1, col=1
            )
        else:
            self.fig.add_trace(
                go.Scatter3d(
                    x=xb0, y=yb0, z=zb0,
                    mode="lines",
                    line=dict(width=line_width, color=color),
                    name=f"{name}-bones",
                    showlegend=True,
                )
            )
            self.fig.add_trace(
                go.Scatter3d(
                    x=f0[:, 0], y=f0[:, 1], z=f0[:, 2],
                    mode="markers",
                    marker=dict(size=marker_size, color=color),
                    name=f"{name}-joints",
                    showlegend=True,
                )
            )

        # 2D traces (right), only if enabled + provided
        if self.show_2d and (K2 is not None):
            f02 = K2[0]
            edges2d_use = edges2d or edges
            ub0, vb0 = self._bones_uv(f02, edges2d_use)

            self.fig.add_trace(
                go.Scatter(
                    x=ub0, y=vb0,
                    mode="lines",
                    line=dict(width=line_width_2d, color=color2d),
                    name=f"{name}-2d-bones",
                    showlegend=True,
                ),
                row=1, col=2
            )
            self.fig.add_trace(
                go.Scatter(
                    x=f02[:, 0], y=f02[:, 1],
                    mode="markers",
                    marker=dict(size=marker_size_2d, color=color2d),
                    name=f"{name}-2d-joints",
                    showlegend=True,
                ),
                row=1, col=2
            )

        # Update fixed axes + rebuild frames for all sequences
        self._recompute_scene_ranges()
        self._recompute_2d_ranges()
        self._rebuild_animation()

        return self.fig

# data = np.load("./walking/motion_pred_3d_Wan2.2-TI2V-5B_frame0000_walking.npz", allow_pickle=True)
# data = np.load("./running_t-20_dit-1_sm/motion_pred_3d_Wan2.2-TI2V-5B_frame0000_running.npz", allow_pickle=True)
# data = np.load("./walking_from_video/motion_pred_3d_Wan2.2-TI2V-5B_Walking_cam_0_render_walking_from_video.npz", allow_pickle=True)
data = np.load("./walking_from_video_single_sample/motion_pred_3d_Wan2.2-TI2V-5B_Walking_cam_0_render_walking_from_video.npz", allow_pickle=True)
# data = np.load("./walking_from_video_t-20_dit-1/motion_pred_3d_Wan2.2-TI2V-5B_Walking_cam_0_render_walking_from_video.npz", allow_pickle=True)
print(data.files)
motion_pred_2d = data["motion_pred_2d"]  # T, J, 2
motion_pred_3d = data["motion_pred_3d"]  # T, J, 3
print(motion_pred_2d.min(), motion_pred_2d.max())
print(motion_pred_3d.shape, motion_pred_2d.shape)
X = motion_pred_3d  # T, J, 3
plot = MultiSkeleton2D3DAnimator(fps=30)
plot.add_sequence(X, K2=motion_pred_2d, edges=None, color="blue", name="Running")



['motion_pred_3d', 'motion_pred_2d']
0.30296904 0.90123725
(61, 65, 3) (61, 65, 2)


In [33]:
x = np.array([1000.0000,  999.7998,  999.5994,  999.3986,  999.1974,  998.9960,
         998.7943,  998.5920,  998.3897,  998.1870,  997.9838,  997.7805,
         997.5768,  997.3727,  997.1682,  996.9636,  996.7586,  996.5530,
         996.3474,  996.1414,  995.9349,  995.7283,  995.5212,  995.3138,
         995.1060,  994.8979,  994.6895,  994.4807,  994.2717,  994.0623,
         993.8524,  993.6423,  993.4319,  993.2211,  993.0098,  992.7984,
         992.5864,  992.3743,  992.1617,  991.9489,  991.7355,  991.5220,
         991.3080,  991.0936,  990.8789,  990.6640,  990.4485,  990.2327,
         990.0167,  989.8002,  989.5833,  989.3661,  989.1487,  988.9307,
         988.7123,  988.4937,  988.2748,  988.0553,  987.8356,  987.6155,
         987.3949,  987.1741,  986.9529,  986.7312,  986.5093,  986.2870,
         986.0642,  985.8411,  985.6176,  985.3937,  985.1695,  984.9448,
         984.7198,  984.4944,  984.2687,  984.0426,  983.8160,  983.5891,
         983.3618,  983.1341,  982.9059,  982.6775,  982.4487,  982.2194,
         981.9897,  981.7597,  981.5291,  981.2984,  981.0672,  980.8356,
         980.6034,  980.3711,  980.1382,  979.9048,  979.6713,  979.4372,
         979.2028,  978.9679,  978.7327,  978.4969,  978.2609,  978.0244,
         977.7874,  977.5501,  977.3124,  977.0742,  976.8356,  976.5967,
         976.3573,  976.1174,  975.8771,  975.6366,  975.3954,  975.1538,
         974.9120,  974.6696,  974.4268,  974.1835,  973.9399,  973.6957,
         973.4514,  973.2064,  972.9611,  972.7151,  972.4691,  972.2222,
         971.9750,  971.7275,  971.4794,  971.2310,  970.9821,  970.7328,
         970.4829,  970.2328,  969.9821,  969.7309,  969.4793,  969.2274,
         968.9749,  968.7218,  968.4685,  968.2147,  967.9603,  967.7055,
         967.4504,  967.1946,  966.9384,  966.6818,  966.4247,  966.1671,
         965.9092,  965.6506,  965.3915,  965.1322,  964.8723,  964.6118,
         964.3510,  964.0897,  963.8278,  963.5655,  963.3028,  963.0394,
         962.7757,  962.5115,  962.2468,  961.9814,  961.7159,  961.4498,
         961.1829,  960.9158,  960.6483,  960.3799,  960.1112,  959.8422,
         959.5726,  959.3023,  959.0316,  958.7606,  958.4887,  958.2167,
         957.9440,  957.6708,  957.3970,  957.1229,  956.8480,  956.5726,
         956.2971,  956.0206,  955.7438,  955.4665,  955.1887,  954.9103,
         954.6314,  954.3520,  954.0720,  953.7914,  953.5104,  953.2289,
         952.9467,  952.6641,  952.3810,  952.0972,  951.8129,  951.5282,
         951.2428,  950.9569,  950.6705,  950.3835,  950.0959,  949.8078,
         949.5192,  949.2300,  948.9403,  948.6500,  948.3591,  948.0676,
         947.7756,  947.4830,  947.1899,  946.8962,  946.6019,  946.3071,
         946.0117,  945.7157,  945.4191,  945.1219,  944.8243,  944.5259,
         944.2270,  943.9276,  943.6274,  943.3268,  943.0256,  942.7237,
         942.4212,  942.1182,  941.8146,  941.5104,  941.2056,  940.9001,
         940.5941,  940.2874,  939.9802,  939.6723,  939.3638,  939.0547,
         938.7451,  938.4346,  938.1237,  937.8122,  937.5000,  937.1872,
         936.8737,  936.5597,  936.2450,  935.9296,  935.6137,  935.2971,
         934.9799,  934.6619,  934.3434,  934.0242,  933.7044,  933.3840,
         933.0629,  932.7411,  932.4187,  932.0956,  931.7719,  931.4475,
         931.1224,  930.7968,  930.4703,  930.1433,  929.8156,  929.4872,
         929.1581,  928.8284,  928.4980,  928.1668,  927.8350,  927.5026,
         927.1694,  926.8356,  926.5010,  926.1658,  925.8299,  925.4932,
         925.1559,  924.8179,  924.4792,  924.1397,  923.7996,  923.4587,
         923.1171,  922.7748,  922.4319,  922.0881,  921.7437,  921.3985,
         921.0526,  920.7060,  920.3586,  920.0106,  919.6617,  919.3122,
         918.9619,  918.6108,  918.2590,  917.9064,  917.5532,  917.1992,
         916.8443,  916.4888,  916.1324,  915.7753,  915.4175,  915.0590,
         914.6996,  914.3394,  913.9785,  913.6168,  913.2543,  912.8911,
         912.5270,  912.1621,  911.7966,  911.4301,  911.0629,  910.6949,
         910.3261,  909.9565,  909.5861,  909.2148,  908.8427,  908.4699,
         908.0963,  907.7218,  907.3465,  906.9704,  906.5934,  906.2156,
         905.8370,  905.4576,  905.0772,  904.6961,  904.3141,  903.9313,
         903.5477,  903.1631,  902.7778,  902.3915,  902.0045,  901.6165,
         901.2277,  900.8380,  900.4474,  900.0560,  899.6637,  899.2704,
         898.8763,  898.4814,  898.0856,  897.6888,  897.2912,  896.8926,
         896.4933,  896.0928,  895.6916,  895.2895,  894.8864,  894.4824,
         894.0775,  893.6716,  893.2648,  892.8571,  892.4485,  892.0389,
         891.6284,  891.2170,  890.8046,  890.3912,  889.9770,  889.5617,
         889.1455,  888.7283,  888.3101,  887.8911,  887.4710,  887.0500,
         886.6279,  886.2049,  885.7809,  885.3558,  884.9299,  884.5029,
         884.0750,  883.6459,  883.2159,  882.7849,  882.3529,  881.9199,
         881.4859,  881.0508,  880.6146,  880.1775,  879.7393,  879.3001,
         878.8599,  878.4186,  877.9762,  877.5328,  877.0883,  876.6427,
         876.1962,  875.7485,  875.2998,  874.8499,  874.3990,  873.9470,
         873.4940,  873.0398,  872.5846,  872.1281,  871.6707,  871.2121,
         870.7524,  870.2916,  869.8297,  869.3666,  868.9025,  868.4371,
         867.9706,  867.5031,  867.0343,  866.5645,  866.0933,  865.6212,
         865.1478,  864.6732,  864.1975,  863.7206,  863.2426,  862.7633,
         862.2829,  861.8013,  861.3184,  860.8344,  860.3491,  859.8627,
         859.3750,  858.8861,  858.3959,  857.9046,  857.4120,  856.9183,
         856.4232,  855.9269,  855.4293,  854.9304,  854.4304,  853.9290,
         853.4265,  852.9225,  852.4173,  851.9108,  851.4031,  850.8940,
         850.3837,  849.8719,  849.3589,  848.8447,  848.3290,  847.8121,
         847.2938,  846.7742,  846.2532,  845.7309,  845.2072,  844.6822,
         844.1558,  843.6281,  843.0990,  842.5684,  842.0366,  841.5032,
         840.9686,  840.4325,  839.8950,  839.3561,  838.8158,  838.2740,
         837.7308,  837.1862,  836.6403,  836.0928,  835.5438,  834.9933,
         834.4414,  833.8881,  833.3333,  832.7771,  832.2193,  831.6600,
         831.0992,  830.5370,  829.9731,  829.4078,  828.8410,  828.2726,
         827.7027,  827.1313,  826.5582,  825.9838,  825.4076,  824.8300,
         824.2507,  823.6698,  823.0875,  822.5034,  821.9178,  821.3306,
         820.7418,  820.1514,  819.5593,  818.9655,  818.3702,  817.7731,
         817.1746,  816.5742,  815.9723,  815.3686,  814.7632,  814.1562,
         813.5475,  812.9371,  812.3250,  811.7110,  811.0955,  810.4782,
         809.8591,  809.2384,  808.6158,  807.9915,  807.3655,  806.7376,
         806.1080,  805.4766,  804.8434,  804.2083,  803.5714,  802.9328,
         802.2922,  801.6499,  801.0058,  800.3597,  799.7119,  799.0621,
         798.4103,  797.7570,  797.1014,  796.4442,  795.7849,  795.1237,
         794.4606,  793.7957,  793.1287,  792.4598,  791.7888,  791.1161,
         790.4412,  789.7643,  789.0856,  788.4047,  787.7219,  787.0370,
         786.3502,  785.6612,  784.9702,  784.2773,  783.5822,  782.8849,
         782.1857,  781.4842,  780.7808,  780.0752,  779.3675,  778.6577,
         777.9456,  777.2314,  776.5152,  775.7966,  775.0761,  774.3531,
         773.6281,  772.9008,  772.1712,  771.4396,  770.7055,  769.9693,
         769.2308,  768.4899,  767.7470,  767.0016,  766.2539,  765.5039,
         764.7515,  763.9969,  763.2399,  762.4805,  761.7188,  760.9546,
         760.1881,  759.4192,  758.6478,  757.8741,  757.0978,  756.3191,
         755.5380,  754.7544,  753.9683,  753.1796,  752.3885,  751.5949,
         750.7987,  750.0001,  749.1987,  748.3949,  747.5884,  746.7794,
         745.9678,  745.1535,  744.3366,  743.5170,  742.6948,  741.8699,
         741.0424,  740.2120,  739.3791,  738.5434,  737.7050,  736.8637,
         736.0197,  735.1730,  734.3234,  733.4711,  732.6160,  731.7579,
         730.8971,  730.0333,  729.1667,  728.2972,  727.4247,  726.5494,
         725.6711,  724.7899,  723.9058,  723.0186,  722.1284,  721.2352,
         720.3389,  719.4398,  718.5374,  717.6321,  716.7236,  715.8120,
         714.8973,  713.9794,  713.0584,  712.1343,  711.2069,  710.2764,
         709.3426,  708.4056,  707.4653,  706.5217,  705.5750,  704.6248,
         703.6713,  702.7145,  701.7544,  700.7909,  699.8240,  698.8536,
         697.8799,  696.9026,  695.9220,  694.9378,  693.9502,  692.9590,
         691.9643,  690.9660,  689.9642,  688.9587,  687.9497,  686.9370,
         685.9206,  684.9006,  683.8768,  682.8494,  681.8182,  680.7833,
         679.7446,  678.7020,  677.6557,  676.6055,  675.5515,  674.4936,
         673.4318,  672.3660,  671.2963,  670.2227,  669.1450,  668.0633,
         666.9777,  665.8879,  664.7939,  663.6961,  662.5940,  661.4877,
         660.3774,  659.2628,  658.1440,  657.0209,  655.8936,  654.7619,
         653.6260,  652.4857,  651.3410,  650.1920,  649.0385,  647.8806,
         646.7181,  645.5513,  644.3799,  643.2039,  642.0233,  640.8382,
         639.6485,  638.4540,  637.2549,  636.0511,  634.8425,  633.6292,
         632.4111,  631.1882,  629.9603,  628.7277,  627.4901,  626.2475,
         625.0000,  623.7475,  622.4899,  621.2274,  619.9597,  618.6868,
         617.4089,  616.1257,  614.8374,  613.5438,  612.2449,  610.9407,
         609.6312,  608.3162,  606.9959,  605.6701,  604.3388,  603.0021,
         601.6598,  600.3118,  598.9583,  597.5991,  596.2343,  594.8638,
         593.4874,  592.1052,  590.7173,  589.3235,  587.9238,  586.5181,
         585.1064,  583.6887,  582.2650,  580.8351,  579.3992,  577.9570,
         576.5087,  575.0541,  573.5931,  572.1258,  570.6522,  569.1721,
         567.6855,  566.1926,  564.6930,  563.1868,  561.6741,  560.1545,
         558.6282,  557.0953,  555.5556,  554.0090,  552.4554,  550.8948,
         549.3274,  547.7529,  546.1712,  544.5824,  542.9865,  541.3832,
         539.7728,  538.1549,  536.5297,  534.8970,  533.2569,  531.6092,
         529.9540,  528.2910,  526.6204,  524.9420,  523.2558,  521.5618,
         519.8598,  518.1499,  516.4320,  514.7059,  512.9717,  511.2293,
         509.4787,  507.7197,  505.9524,  504.1766,  502.3924,  500.5995,
         498.7981,  496.9880,  495.1691,  493.3414,  491.5049,  489.6594,
         487.8049,  485.9413,  484.0687,  482.1867,  480.2956,  478.3951,
         476.4852,  474.5658,  472.6368,  470.6983,  468.7500,  466.7920,
         464.8241,  462.8463,  460.8586,  458.8607,  456.8528,  454.8346,
         452.8061,  450.7673,  448.7180,  446.6581,  444.5876,  442.5065,
         440.4146,  438.3117,  436.1979,  434.0731,  431.9372,  429.7901,
         427.6316,  425.4618,  423.2804,  421.0875,  418.8830,  416.6667,
         414.4385,  412.1984,  409.9462,  407.6820,  405.4054,  403.1165,
         400.8152,  398.5014,  396.1749,  393.8356,  391.4835,  389.1185,
         386.7404,  384.3491,  381.9445,  379.5265,  377.0950,  374.6499,
         372.1910,  369.7183,  367.2316,  364.7309,  362.2159,  359.6866,
         357.1429,  354.5845,  352.0115,  349.4236,  346.8208,  344.2029,
         341.5698,  338.9213,  336.2573,  333.5777,  330.8824,  328.1711,
         325.4438,  322.7003,  319.9405,  317.1642,  314.3713,  311.5616,
         308.7350,  305.8912,  303.0303,  300.1520,  297.2561,  294.3425,
         291.4111,  288.4615,  285.4938,  282.5078,  279.5031,  276.4797,
         273.4375,  270.3762,  267.2956,  264.1956,  261.0760,  257.9365,
         254.7771,  251.5974,  248.3974,  245.1769,  241.9355,  238.6731,
         235.3896,  232.0847,  228.7582,  225.4098,  222.0395,  218.6469,
         215.2318,  211.7941,  208.3333,  204.8495,  201.3423,  197.8115,
         194.2568,  190.6780,  187.0748,  183.4471,  179.7945,  176.1168,
         172.4138,  168.6851,  164.9306,  161.1498,  157.3427,  153.5088,
         149.6479,  145.7597,  141.8440,  137.9004,  133.9286,  129.9283,
         125.8993,  121.8412,  117.7536,  113.6364,  109.4891,  105.3114,
         101.1029,   96.8635,   92.5926,   88.2900,   83.9552,   79.5880,
          75.1880,   70.7547,   66.2879,   61.7871,   57.2519,   52.6820,
          48.0769,   43.4363,   38.7597,   34.0467,   29.2969,   24.5098,
          19.6850,   14.8221,    9.9206,    4.9801])
x[-20]

92.5926